In [36]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import pandas as pd
import seaborn as sns
import matplotlib
from sklearn.model_selection import cross_val_score

from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import learning_curve
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [37]:
# def read_data(folders): 
#     X = []
#     for folder in folders:
#         files_s = glob.glob(f"../raw_data/{folder}/*all*.tif")
#         for file in files_s:
#             filename = os.path.basename(f"../raw_data/{folder}/{file}")
#             file_id, date = filename.split('_all_bands_')
#             date = date.replace('.tif','')
            
#             img = rasterio.open(file)
#             # Compute ndvi mean, median, std
#             ndvi = (img.read(8)-img.read(4))/(img.read(8) + img.read(4))
#             ndvi_ = np.nan_to_num(ndvi, nan=-1)
#             ndvi_values = np.array([x for x in ndvi_.flatten() if x != -1])
#             ndvi_mean = ndvi_values.mean()
#             ndvi_median = np.median(ndvi_values)
#             ndvi_std = np.std(ndvi_values)
            
#             # Compute mi mean, median, std
#             mi = (img.read(8)-img.read(11))/(img.read(8) + img.read(11))
#             mi_ = np.nan_to_num(mi, nan=-1)
#             mi_values =np.array([x for x in mi_.flatten() if x != -1])
#             mi_mean = mi_values.mean()
#             mi_median = np.median(mi_values)
#             mi_std = np.std(mi_values)
            
#             # Compute Cb1 mean, median, std
#             band_combo_1 = img.read(7) + img.read(6) + img.read(4)
#             bc1_ = np.nan_to_num(band_combo_1, nan=-1)
#             bc1_values =np.array([x for x in bc1_.flatten() if x != -1])
#             bc1_mean = bc1_values.mean()
#             bc1_median = np.median(bc1_values)
#             bc1_std = np.std(bc1_values)
            
#             # Compute Cb2 mean, median, std
#             band_combo_2 = img.read(11) + img.read(8) + img.read(2)
#             bc2_ = np.nan_to_num(band_combo_2, nan=-1)
#             bc2_values =np.array([x for x in bc2_.flatten() if x != -1])
#             bc2_mean= bc2_values.mean()
#             bc2_median= np.median(bc2_values)
#             bc2_std= np.std(bc2_values)
            
#             # Compute Cb3 mean, median, std
#             band_combo_3 = (img.read(3)-img.read(8))/(img.read(3)+img.read(8))
#             bc3_ = np.nan_to_num(band_combo_3, nan=-1)
#             bc3_values =np.array([x for x in bc3_.flatten() if x != -1])
#             bc3_mean= bc3_values.mean()
#             bc3_median= np.median(bc3_values)
#             bc3_std= np.std(bc3_values)
            
            
#             b_dict = {'date':date,'file_id':file_id,'target': folder,\
#                       'ndvi_mean': ndvi_mean,'ndvi_median': ndvi_median,'ndvi_std': ndvi_std,\
#                       'mi_mean': mi_mean,'mi_median': mi_median,'mi_std': mi_std,\
#                       'bc1_mean': bc1_mean,'bc1_median': bc1_median,'bc1_std': bc1_std,\
#                       'bc2_mean': bc2_mean,'bc2_median': bc2_median,'bc2_std': bc2_std,\
#                       'bc3_mean': bc3_mean,'bc3_median': bc3_median,'bc3_std': bc3_std}
#             for band in range(1,14):
# #               b_dict[f'b{band}'] = img.read(band).flatten()
#                 b_dict[f'b{band}_mean'] = img.read(band).flatten().mean()
#                 b_dict[f'b{band}_median'] = np.median(img.read(band).flatten())
#                 b_dict[f'b{band}_std'] = np.std(img.read(band).flatten())
            
#             X.append(b_dict)
#     return pd.DataFrame(X)

In [38]:
df = read_data(['rice_1y', 'sugarcane'])
df

<ipython-input-37-d44e22eee1b6>:12: RuntimeWarning: invalid value encountered in true_divide
  ndvi = (img.read(8)-img.read(4))/(img.read(8) + img.read(4))
<ipython-input-37-d44e22eee1b6>:20: RuntimeWarning: invalid value encountered in true_divide
  mi = (img.read(8)-img.read(11))/(img.read(8) + img.read(11))
<ipython-input-37-d44e22eee1b6>:44: RuntimeWarning: invalid value encountered in true_divide
  band_combo_3 = (img.read(3)-img.read(8))/(img.read(3)+img.read(8))


,date,file_id,target,ndvi_mean,ndvi_median,ndvi_std,mi_mean,mi_median,mi_std,bc1_mean,...,b10_std,b11_mean,b11_median,b11_std,b12_mean,b12_median,b12_std,b13_mean,b13_median,b13_std
0,2020-07-29,id_6094bb4df85c5e001b870910,rice_1y,0.479770,0.496448,0.059450,0.893525,0.160805,2.713326,2491.084034,...,2339.127591,871.579832,0.0,1210.098531,583.731092,0.0,819.077788,0.348739,0.0,0.476571
1,2020-12-11,id_60a09545f85c5e001b8714ba,rice_1y,0.024270,0.023761,0.004722,0.324392,0.324265,0.004796,4878.819959,...,7077.375891,824.973251,0.0,1347.785619,880.080247,0.0,1437.700117,0.272634,0.0,0.445314
2,2020-07-29,id_6094f7b0f85c5e001b870a58,rice_1y,0.515295,0.505023,0.039212,0.278630,0.272507,0.028458,2694.798611,...,997.714527,647.715278,902.5,439.888049,407.208333,583.0,274.959085,0.687500,1.0,0.463512
3,2020-08-01,Name_5ea3fd7f1d99bb00101aff6e,rice_1y,0.036236,0.036508,0.001827,0.605546,0.605525,0.001309,8010.869565,...,2384.862697,699.121739,0.0,952.751828,760.582609,0.0,1036.504472,0.350000,0.0,0.476970
4,2021-05-05,id_6077a741f85c5e001b86fd52,rice_1y,0.163137,0.158295,0.058246,0.330438,0.378871,0.122807,1540.683946,...,465.570648,285.217391,293.0,309.532752,221.709030,232.0,235.653334,0.570234,1.0,0.495043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9281,2021-06-03,id_6054a1b558a7cd00480009e9,sugarcane,0.335101,0.283135,0.125234,12.600606,13.740725,3.753553,3646.161905,...,982.226697,2267.308333,2890.0,1247.778053,1798.857143,2245.0,1000.393515,0.770238,1.0,0.420680
9282,2021-03-12,Name_5f2a4ef2868954001c94d1d5,sugarcane,0.230155,0.237195,0.028713,10.142110,9.997819,0.842387,4901.142857,...,1085.471758,3091.312500,3845.5,1620.231608,2508.214286,3095.0,1319.251367,0.794643,1.0,0.403962
9283,2021-01-26,id_6054a1b558a7cd00480009d7,sugarcane,0.230022,0.226676,0.017743,8.482323,8.282961,0.707136,4944.941905,...,1784.732313,2609.014286,4034.0,2112.779774,1805.988571,2830.0,1462.584626,0.606667,1.0,0.488490
9284,2021-06-23,id_6054a1b558a7cd00480009e3,sugarcane,0.375192,0.381902,0.029054,0.646592,0.069275,2.478642,4722.270769,...,3917.839619,1763.051282,2813.0,1414.207416,1305.328205,2056.0,1047.888122,0.609231,1.0,0.487923


In [39]:
df[df['target'] == 'sugarcane'].count()

date           6125
file_id        6125
target         6125
ndvi_mean      6125
ndvi_median    6125
ndvi_std       6125
mi_mean        6125
mi_median      6125
mi_std         6125
bc1_mean       6125
bc1_median     6125
bc1_std        6125
bc2_mean       6125
bc2_median     6125
bc2_std        6125
bc3_mean       6125
bc3_median     6125
bc3_std        6125
b1_mean        6125
b1_median      6125
b1_std         6125
b2_mean        6125
b2_median      6125
b2_std         6125
b3_mean        6125
b3_median      6125
b3_std         6125
b4_mean        6125
b4_median      6125
b4_std         6125
b5_mean        6125
b5_median      6125
b5_std         6125
b6_mean        6125
b6_median      6125
b6_std         6125
b7_mean        6125
b7_median      6125
b7_std         6125
b8_mean        6125
b8_median      6125
b8_std         6125
b9_mean        6125
b9_median      6125
b9_std         6125
b10_mean       6125
b10_median     6125
b10_std        6125
b11_mean       6125
b11_median     6125


In [40]:
df[df['target'] == 'rice_1y'].count()

date           3161
file_id        3161
target         3161
ndvi_mean      3161
ndvi_median    3161
ndvi_std       3161
mi_mean        3161
mi_median      3161
mi_std         3161
bc1_mean       3161
bc1_median     3161
bc1_std        3161
bc2_mean       3161
bc2_median     3161
bc2_std        3161
bc3_mean       3161
bc3_median     3161
bc3_std        3161
b1_mean        3161
b1_median      3161
b1_std         3161
b2_mean        3161
b2_median      3161
b2_std         3161
b3_mean        3161
b3_median      3161
b3_std         3161
b4_mean        3161
b4_median      3161
b4_std         3161
b5_mean        3161
b5_median      3161
b5_std         3161
b6_mean        3161
b6_median      3161
b6_std         3161
b7_mean        3161
b7_median      3161
b7_std         3161
b8_mean        3161
b8_median      3161
b8_std         3161
b9_mean        3161
b9_median      3161
b9_std         3161
b10_mean       3161
b10_median     3161
b10_std        3161
b11_mean       3161
b11_median     3161


In [10]:
df.keys()

Index(['date', 'file_id', 'target', 'ndvi_mean', 'ndvi_median', 'ndvi_std',
       'mi_mean', 'mi_median', 'mi_std', 'bc1_mean', 'bc1_median', 'bc1_std',
       'bc2_mean', 'bc2_median', 'bc2_std', 'bc3_mean', 'bc3_median',
       'bc3_std', 'b1_mean', 'b1_median', 'b1_std', 'b2_mean', 'b2_median',
       'b2_std', 'b3_mean', 'b3_median', 'b3_std', 'b4_mean', 'b4_median',
       'b4_std', 'b5_mean', 'b5_median', 'b5_std', 'b6_mean', 'b6_median',
       'b6_std', 'b7_mean', 'b7_median', 'b7_std', 'b8_mean', 'b8_median',
       'b8_std', 'b9_mean', 'b9_median', 'b9_std', 'b10_mean', 'b10_median',
       'b10_std', 'b11_mean', 'b11_median', 'b11_std', 'b12_mean',
       'b12_median', 'b12_std', 'b13_mean', 'b13_median', 'b13_std'],
      dtype='object')

In [41]:
df['month'] = pd.DatetimeIndex(df['date']).month
#     df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df = df.sort_values(by=['date'])
df

,date,file_id,target,ndvi_mean,ndvi_median,ndvi_std,mi_mean,mi_median,mi_std,bc1_mean,...,b11_mean,b11_median,b11_std,b12_mean,b12_median,b12_std,b13_mean,b13_median,b13_std,month
1507,2020-07-02,Name_5ea4090c1d99bb00101aff7d,rice_1y,0.287349,0.282344,0.018389,7.265656,7.259265,0.131201,11050.634615,...,4605.673077,4605.0,122.624148,3715.576923,3697.0,126.258335,1.000000,1.0,0.000000,7
2895,2020-07-02,Name_5eaedf92359f990010fd8194,rice_1y,0.066608,0.066404,0.004203,0.393128,0.392827,0.003830,16531.506631,...,2513.098143,2815.0,866.912368,2301.689655,2576.0,793.687046,0.893899,1.0,0.307967,7
2359,2020-07-02,Name_5eaba79a359f990010fd7de2,rice_1y,0.033908,0.033948,0.002177,0.653560,0.653640,0.001437,15000.466667,...,1070.793333,1337.0,535.409031,1350.486667,1687.0,675.262100,0.800000,1.0,0.400000,7
461,2020-07-02,Name_5eaee082359f990010fd819c,rice_1y,0.066670,0.066405,0.004163,0.393181,0.392867,0.003811,16380.453581,...,2490.031830,2811.0,894.504201,2280.591512,2575.0,818.977468,0.885942,1.0,0.317882,7
1940,2020-07-02,Name_5eb6b5df359f990010fd8548,rice_1y,0.216300,0.215431,0.007173,0.382416,0.382183,0.006579,6348.534722,...,1082.703704,1771.0,872.258552,939.983796,1535.0,757.245014,0.606481,1.0,0.488530,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5143,2021-06-30,id_6054a1b458a7cd0048000989,sugarcane,0.427349,0.430541,0.062322,6.145892,9.196321,4.499282,5191.629304,...,2325.835897,3438.0,1668.483726,1629.808791,2357.0,1177.143668,0.662271,1.0,0.472936,6
1574,2021-06-30,Name_5eb66cfe359f990010fd8511,rice_1y,0.709861,0.723051,0.037338,0.357526,0.374225,0.047257,6052.744643,...,1438.671429,1868.0,848.620641,837.151786,1072.0,495.569528,0.744643,1.0,0.436062,6
1135,2021-06-30,Name_5e79c928943c0e0011b03cfa,rice_1y,0.291483,0.257002,0.095828,0.219654,0.239695,0.067831,3580.263736,...,890.106227,1223.0,876.840891,629.655678,938.0,610.242550,0.534799,1.0,0.498788,6
704,2021-06-30,Name_5eb635a7359f990010fd84fa,rice_1y,3.285669,4.065257,1.743955,0.134904,0.086110,0.519370,17718.246032,...,4762.018519,6264.0,2834.424509,4039.566138,5361.0,2404.070352,0.740741,1.0,0.438228,6


In [42]:
df.to_csv('dataframe5.csv', index=False)

In [43]:
data = pd.read_csv('dataframe5.csv')
data

,date,file_id,target,ndvi_mean,ndvi_median,ndvi_std,mi_mean,mi_median,mi_std,bc1_mean,...,b11_mean,b11_median,b11_std,b12_mean,b12_median,b12_std,b13_mean,b13_median,b13_std,month
0,2020-07-02,Name_5ea4090c1d99bb00101aff7d,rice_1y,0.287349,0.282344,0.018389,7.265656,7.259265,0.131201,11050.634615,...,4605.673077,4605.0,122.624148,3715.576923,3697.0,126.258335,1.000000,1.0,0.000000,7
1,2020-07-02,Name_5eaedf92359f990010fd8194,rice_1y,0.066608,0.066404,0.004203,0.393128,0.392827,0.003830,16531.506631,...,2513.098143,2815.0,866.912368,2301.689655,2576.0,793.687046,0.893899,1.0,0.307967,7
2,2020-07-02,Name_5eaba79a359f990010fd7de2,rice_1y,0.033908,0.033948,0.002177,0.653560,0.653640,0.001437,15000.466667,...,1070.793333,1337.0,535.409031,1350.486667,1687.0,675.262100,0.800000,1.0,0.400000,7
3,2020-07-02,Name_5eaee082359f990010fd819c,rice_1y,0.066670,0.066405,0.004163,0.393181,0.392867,0.003811,16380.453581,...,2490.031830,2811.0,894.504201,2280.591512,2575.0,818.977468,0.885942,1.0,0.317882,7
4,2020-07-02,Name_5eb6b5df359f990010fd8548,rice_1y,0.216300,0.215431,0.007173,0.382416,0.382183,0.006579,6348.534722,...,1082.703704,1771.0,872.258552,939.983796,1535.0,757.245014,0.606481,1.0,0.488530,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9281,2021-06-30,id_6054a1b458a7cd0048000989,sugarcane,0.427349,0.430541,0.062322,6.145892,9.196321,4.499282,5191.629304,...,2325.835897,3438.0,1668.483726,1629.808791,2357.0,1177.143668,0.662271,1.0,0.472936,6
9282,2021-06-30,Name_5eb66cfe359f990010fd8511,rice_1y,0.709861,0.723051,0.037338,0.357526,0.374225,0.047257,6052.744643,...,1438.671429,1868.0,848.620641,837.151786,1072.0,495.569528,0.744643,1.0,0.436062,6
9283,2021-06-30,Name_5e79c928943c0e0011b03cfa,rice_1y,0.291483,0.257002,0.095828,0.219654,0.239695,0.067831,3580.263736,...,890.106227,1223.0,876.840891,629.655678,938.0,610.242550,0.534799,1.0,0.498788,6
9284,2021-06-30,Name_5eb635a7359f990010fd84fa,rice_1y,3.285669,4.065257,1.743955,0.134904,0.086110,0.519370,17718.246032,...,4762.018519,6264.0,2834.424509,4039.566138,5361.0,2404.070352,0.740741,1.0,0.438228,6


In [44]:
df2 = data.copy()

In [45]:
#GroupShuffleSplit
from sklearn.model_selection import GroupShuffleSplit
train_inds, test_inds = next(GroupShuffleSplit(test_size=.20, n_splits=2, random_state = 7).split(df2, groups=df2['file_id']))
train = df2.iloc[train_inds]
test = df2.iloc[test_inds]

In [46]:
print(train.shape, test.shape)

(7328, 58) (1958, 58)


In [47]:
A = train.file_id.unique()
B = test.file_id.unique()
bool(set(A).intersection(B))

False

In [48]:
X_train = train.drop(columns = ['target','date','file_id'])
y_train= train['target']

X_test = test.drop(columns = ['target','date','file_id'])
y_test = test['target']

In [49]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

pipe = Pipeline([('scaler', RobustScaler()),('pca', PCA()),('knn', KNeighborsClassifier(n_neighbors=4, leaf_size = 1, p = 1))])
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

pipe.score(X_test,y_test)

print('Accuracy',accuracy_score(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print(cm)

Accuracy 0.8273748723186926
[[ 535  121]
 [ 217 1085]]


In [50]:
# Save model 
import joblib
# save the model to disk
filename ='pipeline_test5.joblib'
joblib.dump(pipe,filename)
print("saved pipeline.joblib locally")

saved pipeline.joblib locally


In [51]:
class0_df = read_data(['class0'])
class0_df['month'] = pd.DatetimeIndex(class0_df['date']).month
class0_df = class0_df.sort_values(by=['date'])
class0_df

<ipython-input-37-d44e22eee1b6>:12: RuntimeWarning: invalid value encountered in true_divide
  ndvi = (img.read(8)-img.read(4))/(img.read(8) + img.read(4))
<ipython-input-37-d44e22eee1b6>:20: RuntimeWarning: invalid value encountered in true_divide
  mi = (img.read(8)-img.read(11))/(img.read(8) + img.read(11))
<ipython-input-37-d44e22eee1b6>:44: RuntimeWarning: invalid value encountered in true_divide
  band_combo_3 = (img.read(3)-img.read(8))/(img.read(3)+img.read(8))


,date,file_id,target,ndvi_mean,ndvi_median,ndvi_std,mi_mean,mi_median,mi_std,bc1_mean,...,b11_mean,b11_median,b11_std,b12_mean,b12_median,b12_std,b13_mean,b13_median,b13_std,month
111,2020-07-03,Name_5e70651d943c0e0011b03a40,class0,0.507465,0.518372,0.042512,0.242774,0.252075,0.033055,4817.093985,...,1306.387218,2262.0,1168.146120,871.030075,1500.0,779.938916,0.556391,1.0,0.496810,7
90,2020-07-04,Name_5e4f83e6ec844300113b9d6c,class0,0.029271,0.029265,0.001718,0.684588,0.684451,0.000973,20470.277778,...,1343.041667,1930.0,890.880517,1770.125000,2546.0,1174.180919,0.694444,1.0,0.460642,7
103,2020-07-04,Name_5e4e5b6cec844300113b9cff,class0,0.028881,0.028881,0.000814,0.691464,0.691464,0.000729,29250.000000,...,1871.500000,1871.5,4.500000,2487.000000,2487.0,1.000000,1.000000,1.0,0.000000,7
256,2020-07-05,Name_5e599fc31aed8b0010e82911,class0,0.099210,0.099210,0.004888,6.919127,6.919127,6.912903,6799.000000,...,2358.000000,2358.0,43.000000,2143.500000,2143.5,82.500000,1.000000,1.0,0.000000,7
273,2020-07-05,Name_5e5e03fbd370860011cf1cb0,class0,0.080458,0.082173,0.007211,0.042807,0.039465,0.008960,6611.333333,...,2189.444444,2792.0,1171.384946,1882.888889,2413.0,1006.674111,0.777778,1.0,0.415740,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,2021-06-30,Name_5e599fc31aed8b0010e82911,class0,12.088203,12.088203,12.039927,20.016913,20.016913,0.705008,4891.000000,...,1889.500000,1889.5,114.500000,1677.000000,1677.0,114.000000,1.000000,1.0,0.000000,6
265,2021-06-30,Name_5e541e341aed8b0010e82776,class0,0.179475,0.024426,0.885100,0.135964,0.136143,0.014831,11858.627381,...,3026.721429,4214.0,1971.593304,2129.852381,2962.0,1388.679204,0.703571,1.0,0.456682,6
115,2021-06-30,Name_5e70651d943c0e0011b03a40,class0,0.255113,0.254026,0.012351,0.104361,0.105240,0.009355,7138.112782,...,2257.819549,3048.0,1363.915228,1989.488722,2669.0,1203.489033,0.733083,1.0,0.442349,6
246,2021-06-30,Name_5e5e03fbd370860011cf1cb0,class0,1.284851,0.049324,3.040082,2.632454,0.103160,4.013117,7251.111111,...,2184.222222,2383.0,1303.055535,1970.000000,2288.0,1247.832521,0.777778,1.0,0.415740,6


In [53]:
class0_X = class0_df.drop(columns = ['date', 'file_id','target'])
class0_X

,ndvi_mean,ndvi_median,ndvi_std,mi_mean,mi_median,mi_std,bc1_mean,bc1_median,bc1_std,bc2_mean,...,b11_mean,b11_median,b11_std,b12_mean,b12_median,b12_std,b13_mean,b13_median,b13_std,month
111,0.507465,0.518372,0.042512,0.242774,0.252075,0.033055,4817.093985,8403.0,4304.507279,4171.928571,...,1306.387218,2262.0,1168.146120,871.030075,1500.0,779.938916,0.556391,1.0,0.496810,7
90,0.029271,0.029265,0.001718,0.684588,0.684451,0.000973,20470.277778,29457.0,13578.472711,15108.375000,...,1343.041667,1930.0,890.880517,1770.125000,2546.0,1174.180919,0.694444,1.0,0.460642,7
103,0.028881,0.028881,0.000814,0.691464,0.691464,0.000729,29250.000000,29250.0,21.000000,21555.500000,...,1871.500000,1871.5,4.500000,2487.000000,2487.0,1.000000,1.000000,1.0,0.000000,7
256,0.099210,0.099210,0.004888,6.919127,6.919127,6.912903,6799.000000,6799.0,139.000000,6680.000000,...,2358.000000,2358.0,43.000000,2143.500000,2143.5,82.500000,1.000000,1.0,0.000000,7
273,0.080458,0.082173,0.007211,0.042807,0.039465,0.008960,6611.333333,8441.0,3537.034665,6719.222222,...,2189.444444,2792.0,1171.384946,1882.888889,2413.0,1006.674111,0.777778,1.0,0.415740,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,12.088203,12.088203,12.039927,20.016913,20.016913,0.705008,4891.000000,4891.0,463.000000,4515.500000,...,1889.500000,1889.5,114.500000,1677.000000,1677.0,114.000000,1.000000,1.0,0.000000,6
265,0.179475,0.024426,0.885100,0.135964,0.136143,0.014831,11858.627381,16439.5,7740.857877,11320.169048,...,3026.721429,4214.0,1971.593304,2129.852381,2962.0,1388.679204,0.703571,1.0,0.456682,6
115,0.255113,0.254026,0.012351,0.104361,0.105240,0.009355,7138.112782,9491.5,4322.164264,6677.496241,...,2257.819549,3048.0,1363.915228,1989.488722,2669.0,1203.489033,0.733083,1.0,0.442349,6
246,1.284851,0.049324,3.040082,2.632454,0.103160,4.013117,7251.111111,7467.0,4401.441189,7254.222222,...,2184.222222,2383.0,1303.055535,1970.000000,2288.0,1247.832521,0.777778,1.0,0.415740,6


In [54]:
y_pred_class0 = pipe.predict(class0_X)
y_pred_class0

array(['sugarcane', 'rice_1y', 'rice_1y', 'rice_1y', 'sugarcane',
       'rice_1y', 'sugarcane', 'rice_1y', 'sugarcane', 'rice_1y',
       'rice_1y', 'sugarcane', 'rice_1y', 'sugarcane', 'sugarcane',
       'sugarcane', 'sugarcane', 'rice_1y', 'sugarcane', 'rice_1y',
       'rice_1y', 'rice_1y', 'rice_1y', 'sugarcane', 'rice_1y', 'rice_1y',
       'rice_1y', 'sugarcane', 'rice_1y', 'sugarcane', 'sugarcane',
       'sugarcane', 'rice_1y', 'sugarcane', 'sugarcane', 'rice_1y',
       'rice_1y', 'sugarcane', 'rice_1y', 'rice_1y', 'rice_1y', 'rice_1y',
       'rice_1y', 'sugarcane', 'rice_1y', 'rice_1y', 'sugarcane',
       'rice_1y', 'rice_1y', 'rice_1y', 'rice_1y', 'rice_1y', 'sugarcane',
       'rice_1y', 'rice_1y', 'rice_1y', 'rice_1y', 'rice_1y', 'rice_1y',
       'rice_1y', 'sugarcane', 'sugarcane', 'rice_1y', 'sugarcane',
       'rice_1y', 'rice_1y', 'rice_1y', 'rice_1y', 'rice_1y', 'sugarcane',
       'rice_1y', 'sugarcane', 'rice_1y', 'sugarcane', 'rice_1y',
       'sugarcane', 'ri

In [55]:
import numpy as np
(unique, counts) = np.unique(y_pred_class0, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([['rice_1y', 203],
       ['sugarcane', 103]], dtype=object)

In [56]:
class1_df = read_data(['class1'])
class1_df['month'] = pd.DatetimeIndex(class1_df['date']).month
class1_df = class1_df.sort_values(by=['date'])
class1_df

<ipython-input-37-d44e22eee1b6>:12: RuntimeWarning: invalid value encountered in true_divide
  ndvi = (img.read(8)-img.read(4))/(img.read(8) + img.read(4))
<ipython-input-37-d44e22eee1b6>:20: RuntimeWarning: invalid value encountered in true_divide
  mi = (img.read(8)-img.read(11))/(img.read(8) + img.read(11))
<ipython-input-37-d44e22eee1b6>:44: RuntimeWarning: invalid value encountered in true_divide
  band_combo_3 = (img.read(3)-img.read(8))/(img.read(3)+img.read(8))


,date,file_id,target,ndvi_mean,ndvi_median,ndvi_std,mi_mean,mi_median,mi_std,bc1_mean,...,b11_mean,b11_median,b11_std,b12_mean,b12_median,b12_std,b13_mean,b13_median,b13_std,month
251,2020-07-03,Name_5ec4a0aa70bb79001069c2e5,class1,0.084445,0.084221,0.006810,0.224823,0.225494,0.005684,9866.475000,...,2179.531667,2788.0,1163.687359,1717.061667,2198.0,916.722287,0.778333,1.0,0.415368,7
96,2020-07-03,Name_5e87e77d62e2380012e08aae,class1,0.154198,0.154774,0.006170,0.269663,0.270553,0.004386,2208.082230,...,470.486411,0.0,1014.523917,382.928223,0.0,825.723196,0.177003,0.0,0.381672,7
68,2020-07-03,Name_5ec1684c70bb79001069c0b7,class1,0.037368,0.037440,0.006641,0.259543,0.260314,0.008382,9150.726293,...,1818.371767,3441.0,1756.924984,1424.921336,2669.0,1376.921442,0.517241,1.0,0.499703,7
209,2020-07-03,Name_5ec4a0a870bb79001069c2e3,class1,0.039833,0.039638,0.007082,0.261717,0.262272,0.007194,6623.608004,...,1318.404605,0.0,1698.276518,1031.322917,0.0,1328.559311,0.376096,0.0,0.484405,7
273,2020-07-03,Name_5e90a73162e2380012e08dcd,class1,0.153834,0.154396,0.006261,0.269064,0.269919,0.004421,2413.291248,...,514.670014,0.0,1050.112999,418.991392,0.0,854.897358,0.193687,0.0,0.395187,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,2021-06-30,Name_5eaeb311359f990010fd8151,class1,0.274174,0.288259,0.086654,7.124731,11.235254,5.671536,7000.125000,...,2843.916667,2955.0,597.172778,2201.250000,2236.5,492.556363,0.958333,1.0,0.199826,6
203,2021-06-30,Name_5eb68c2b359f990010fd8526,class1,2.915236,4.410014,2.343150,0.127597,0.060332,0.622469,15203.843195,...,4270.053254,5795.0,2685.863352,3668.642012,4941.0,2310.761773,0.718935,1.0,0.449519,6
138,2021-06-30,Name_5eba1177359f990010fd8756,class1,2.154192,0.050406,2.289934,0.130658,0.124669,0.026823,14289.181818,...,3725.156364,5053.0,2572.773905,3021.581818,4099.0,2095.328018,0.683636,1.0,0.465057,6
403,2021-06-30,Name_5eaf8f8d359f990010fd81f9,class1,0.137557,0.128179,0.045458,4.821027,0.092465,6.295254,7520.333333,...,2687.533333,2554.0,212.764930,2225.533333,2228.0,186.620423,1.000000,1.0,0.000000,6


In [57]:
class1_X = class1_df.drop(columns = ['date', 'file_id','target'])
y_pred_class1 = pipe.predict(class1_X)
y_pred_class1

array(['sugarcane', 'rice_1y', 'sugarcane', 'rice_1y', 'rice_1y',
       'sugarcane', 'rice_1y', 'sugarcane', 'rice_1y', 'sugarcane',
       'sugarcane', 'sugarcane', 'rice_1y', 'rice_1y', 'rice_1y',
       'rice_1y', 'rice_1y', 'sugarcane', 'rice_1y', 'sugarcane',
       'rice_1y', 'sugarcane', 'sugarcane', 'sugarcane', 'rice_1y',
       'rice_1y', 'sugarcane', 'rice_1y', 'rice_1y', 'rice_1y', 'rice_1y',
       'rice_1y', 'sugarcane', 'sugarcane', 'rice_1y', 'rice_1y',
       'sugarcane', 'rice_1y', 'sugarcane', 'rice_1y', 'sugarcane',
       'sugarcane', 'rice_1y', 'sugarcane', 'sugarcane', 'rice_1y',
       'rice_1y', 'rice_1y', 'rice_1y', 'rice_1y', 'rice_1y', 'rice_1y',
       'rice_1y', 'rice_1y', 'rice_1y', 'sugarcane', 'sugarcane',
       'rice_1y', 'rice_1y', 'sugarcane', 'rice_1y', 'rice_1y', 'rice_1y',
       'sugarcane', 'rice_1y', 'sugarcane', 'rice_1y', 'rice_1y',
       'rice_1y', 'rice_1y', 'sugarcane', 'sugarcane', 'rice_1y',
       'rice_1y', 'rice_1y', 'sugarcane', '

In [58]:
import numpy as np
(unique, counts) = np.unique(y_pred_class1, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([['rice_1y', 331],
       ['sugarcane', 129]], dtype=object)